In [1]:
import math
import torch
import time
import random
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.parameter import Parameter
from torch.nn import init
from torch import Tensor
from scipy.special import gamma 
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

#Mean Square Error
def MSE(pred,true):
    return np.mean((pred-true)**2)

#Mean Absolute Error
def MAE(pred, true):
    return np.mean(np.abs(pred-true))

class Fractional_Order_Matrix_Differential_Solver(torch.autograd.Function):
    @staticmethod
    def forward(ctx,input1,w,b,alpha,k,epoch):
        alpha = torch.tensor(alpha)
        k = torch.tensor(k)
        epoch = torch.tensor(epoch)
        ctx.save_for_backward(input1,w,b,alpha,k,epoch)
        outputs = input1@w + b
        return outputs

    @staticmethod
    def backward(ctx, grad_outputs):
        input1,w,b,alpha,k,epoch = ctx.saved_tensors
        x_fractional, w_fractional = Fractional_Order_Matrix_Differential_Solver.Fractional_Order_Matrix_Differential_Linear(input1,w,b,alpha,k,epoch)   
        x_grad = torch.mm(grad_outputs,x_fractional)
        w_grad = torch.mm(w_fractional,grad_outputs)
        b_grad = grad_outputs.sum(dim=0)
        return x_grad, w_grad, b_grad,None,None,None

    @staticmethod
    def Fractional_Order_Matrix_Differential_Linear(x,w,b,alpha,k,epoch):
        #w
        wf = w[:,0].view(1,-1)
        #main
        w_main = torch.mul(x,(torch.abs(wf)+1e-8)**(1-alpha)/gamma(2-alpha))
        #partial
        x_rows, x_cols = x.size()
        bias = torch.full((x_rows, x_cols),b[0].item())
        bias = bias.to(device)
        w_partial = torch.mul(torch.mm(x,wf.T).view(-1,1).expand(-1,x_cols) - torch.mul(x,wf) + bias, torch.sign(wf)*(torch.abs(wf)+1e-8)**(-alpha)/gamma(1-alpha))
        return w.T, (w_main + torch.exp(-k*epoch)*w_partial).T

class FLinear(nn.Module):
    
    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, alpha=0.9, k = 0.9, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.k = k

        self.weight = Parameter(torch.empty((in_features, out_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            init.uniform_(self.bias, -bound, bound)

    def forward(self, x, epoch):
        return Fractional_Order_Matrix_Differential_Solver.apply(x, self.weight, self.bias, self.alpha, self.k, epoch)

    def extra_repr(self) -> str:
        return f"in_features={self.in_features}, out_features={self.out_features}, bias={self.bias is not None}"
    
def split(X,y):
    X_train,X_temp,y_train,y_temp = train_test_split(X,y,test_size=0.3,shuffle=False)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.333,shuffle=False)
    return X_train,X_val,X_test,y_train,y_val,y_test

slide_windows_size = 192  #i.e.,input length 
pred_length = 384     #i.e.,prediction lengths 
stock = 'ETTm2'
df_DJIA = pd.read_csv(r'./data/'+stock+'.csv')
del df_DJIA['date']        #ETT
# scaler = StandardScaler()
scaler = MinMaxScaler(feature_range=(0, 1))

sca_DJIA = scaler.fit_transform(df_DJIA)


features_j = 6     
def create_sequences(data, slide_windows_size, pred_length):
    X, y = [], []
    for i in range(len(data) - slide_windows_size - pred_length + 1):
        X.append(data[i:i+slide_windows_size, :])  # sliding window size [seq_len, features]
        y.append(data[i+slide_windows_size:i+slide_windows_size+pred_length, features_j])  
    return np.array(X), np.array(y)

X, y = create_sequences(sca_DJIA, slide_windows_size, pred_length)
X = torch.Tensor(X).to(device)
y = torch.Tensor(y).to(device)

X_train,X_val,X_test,y_train,y_val,y_test = split(X,y)   #7:2:1 


batch_size = 256
set_seed()
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

# class MLP(nn.Module):
#     def __init__(self, input_size, hidden_size1=256, hidden_size2=128,output_size=pred_length):   ###DJI:hidden_size=256,ETTh1:hidden_size=128.
#         super().__init__()
#         self.flatten = nn.Flatten()
#         self.linear1 = nn.Linear(input_size, hidden_size1)       
#         self.leakrelu1 = nn.LeakyReLU()                          
#         self.linear2 = nn.Linear(hidden_size1, hidden_size2) 
#         self.leakrelu2 = nn.LeakyReLU()
#         self.linear3 = nn.Linear(hidden_size2, output_size)       
#         #self.linear2 = nn.Linear(hidden_size, output_size)                                 #To calculate the Memory usage on SGD.

#     def forward(self, x):
#         x = self.flatten(x)    # (batch_size, seq_len*num_features)
#         x = self.leakrelu1(self.linear1(x))  
#         x = self.leakrelu2(self.linear2(x))
#         x = self.linear3(x)
#         return x
    
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1=256, hidden_size2=128,output_size=pred_length):   ###DJI:hidden_size=256,ETTh1:hidden_size=128.
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear1 = FLinear(input_size, hidden_size1, alpha, k)       
        self.leakrelu1 = nn.LeakyReLU()                          
        self.linear2 = FLinear(hidden_size1, hidden_size2, alpha, k) 
        self.leakrelu2 = nn.LeakyReLU()
        self.linear3 = FLinear(hidden_size2, output_size, alpha, k)       
        #self.linear2 = nn.Linear(hidden_size, output_size)                                 #To calculate the Memory usage on SGD.

    def forward(self, x, epoch=0):
        x = self.flatten(x)    # (batch_size, seq_len*num_features)
        x = self.leakrelu1(self.linear1(x, epoch))  
        x = self.leakrelu2(self.linear2(x, epoch))
        x = self.linear3(x, epoch)
        return x

alpha = 1.0   ####0.7,0.8,0.9,1.0
k = 0.01
num_feature = 7  



peak_memory_max = 0
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
initial_memory = torch.cuda.memory_allocated() / 1024**2

time_start = time.time()
set_seed()
model = MLP(input_size=slide_windows_size*num_feature).to(device)

lr =1e-3
num_epochs = 10   #1500

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# optimizer = torch.optim.SGD(model.parameters(), lr=lr)

for ii in range(num_epochs):
    model.train()
    for batch_idx,(inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()   #The default value of retain_graph is False.
        optimizer.step()
        peak_memory = torch.cuda.max_memory_allocated() / 1024**2

        if peak_memory_max < peak_memory:
            peak_memory_max = peak_memory

time_end = time.time()
print(f"Initial Memory: {initial_memory:.4f} MB")
print(f"Peak Memory: {peak_memory_max:.4f} MB")
print(f"Memory: {(peak_memory_max-initial_memory):.4f} MB")
print(f"Training time: {(time_end-time_start)/10:.4f} s")
###########To ensure fair comparisons, restart the kernel before running each experiment.

Initial Memory: 912.4282 MB
Peak Memory: 945.2031 MB
Memory: 32.7749 MB
Training time: 2.2853 s
